# TO RUN
1. Make sure you installed all the dependencies
2. open **'init_notebook.py'** and change the **projectFolder** to the src folder of the repository.
3. In the following cell change the dataDir to the data directory of the InD dataset

In [1]:
dataDir = "G:AV datasets/inD-dataset-v1.0/data/"
# dataDir = "E:/Datasets/inD-dataset-v1.0/data/"

In [2]:
exec(open('init_notebook.py').read())
%load_ext autoreload
%autoreload 2
%tb

current working dirE:\AV\DataSetExploration\drone-dataset-tools\src


No traceback available to show.


In [3]:
import logging
logging.basicConfig(level=logging.INFO)
from loguru import logger
logger.remove()
logger.add(sys.stdout, format="{level} {message}", filter="my_module", level="INFO")

from extractors.loader import Loader

In [4]:
loader = Loader(dataDir, 'inD')

In [5]:
loc2data = loader.getLocationData(2, useSceneConfigToExtract=True, precomputeSceneData=False)

INFO:root:recordingIds: [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
INFO:root:Downsampling recording 18 from 25.0 to 2.5
downsampling: 100%|█████████████████████████████████████████████████████████| 625989/625989 [00:39<00:00, 15724.01it/s]
INFO:root:Downsampling recording 19 from 25.0 to 2.5
downsampling: 100%|█████████████████████████████████████████████████████████| 492008/492008 [00:31<00:00, 15792.70it/s]
INFO:root:Downsampling recording 20 from 25.0 to 2.5
downsampling: 100%|█████████████████████████████████████████████████████████| 634916/634916 [00:40<00:00, 15842.78it/s]
INFO:root:Downsampling recording 21 from 25.0 to 2.5
downsampling: 100%|█████████████████████████████████████████████████████████| 796178/796178 [00:49<00:00, 16012.03it/s]
INFO:root:Downsampling recording 22 from 25.0 to 2.5
downsampling: 100%|█████████████████████████████████████████████████████████| 753743/753743 [00:47<00:00, 16021.16it/s]
INFO:root:Downsampling recording 23 from 25.0 to 2.5
downsampl

In [6]:
pedData = loc2data.getSceneCrossingDf(11, 5.0, 5.0)


crossing recording: 100%|██████████████████████████████████████████████████████████████| 12/12 [05:24<00:00, 27.07s/it]


In [7]:
otherData = loc2data.getSceneOtherDf(11)

recording-18-scene10-bicycle-Ids: 100%|████████████████████████████████████████████████| 92/92 [00:01<00:00, 73.90it/s]
recording-18-scene10-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene11-bicycle-Ids: 100%|████████████████████████████████████████████████| 92/92 [00:01<00:00, 73.19it/s]
recording-18-scene11-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene20-bicycle-Ids: 100%|████████████████████████████████████████████████| 92/92 [00:01<00:00, 73.07it/s]
recording-18-scene20-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene21-bicycle-Ids: 100%|████████████████████████████████████████████████| 92/92 [00:01<00:00, 71.88it/s]
recording-18-scene21-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene22-bicycle-Ids: 100%|████████████████████████████████████████████████| 92/92 [00:01<00:00, 71.70it/s]
recording-18-scene22-truck_bus-Ids: 0it [00:00, ?it/s]
recording-18-scene30-bicycle-Ids: 100%|████████████████████████████████████████████████| 92/92 [00:01<00:00, 72.50it/s]
recor

In [ ]:
from extractors.SceneData import SceneData
sceneData = SceneData(
    2,
    0.012699935,
    11,
    {
      "centerXPixel": 688.5,
      "centerYPixel": 273.5,
      "centerX": 67.30,
      "centerY": -26.74,
      "angle": 12.17,
      "roadWidth": 5.0,
      "boxWidth": 5.0
    },
    5.0,
    5.0,
    pedData=pedData,
    otherData=otherData,
    backgroundImagePath="G:AV datasets/inD-dataset-v1.0/data/18_background.png"
)

In [ ]:
localPedData = sceneData.getPedDataInSceneCoordinates()
localPedData[localPedData["uniqueTrackId"]==29476]